**CLIENT** Notebook for interacting with Mistral instance running in the "Mistral-core.ipynb" notebook

Substitute URL_GENERATED_BY_ngrok with the URL generated when running ngrok in the "Mistral-core.ipynb" notebook

In [ ]:
import os

url_ollama_host = "URL_GENERATED_BY_ngrok"

os.environ["OLLAMA_HOST"] = url_ollama_host

Install here Ollama (see "Mistral-core.ipynb" notebook for clarification about the following command

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download#https://github.com/jmorganca/ollama/releases/download/v0.1.27#' | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Launch the command to pull Mistral

In [ ]:
!ollama pull mistral

Install dependencies

In [ ]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface

In [ ]:
!pip install llama-index-llms-openai

Create a directory "data" and upload inside the description documents of Atomic Data Services (e.g., JSON OpenAPI specification documents)

In [ ]:
!mkdir ./data

Build the index for **RAG** modality.
Remember to set the base_url variable to the URL generated by ngrok so the core of Mistral is served on the other notebook with GPU RAM.

A retriever is also configured to test the query interaction mode (not detailed in the notebook) with respect to the chat mode

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader("data").load_data()

# bge-m3 embedding model
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# ollama
Settings.llm = Ollama(model="mistral", request_timeout=300.0, base_url=url_ollama_host)

index = VectorStoreIndex.from_documents(
    documents,
)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="compact",
)


In [ ]:
print(Settings.llm)

callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7e4c1bfe3850> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x7e4d1b84f7f0> completion_to_prompt=<function default_completion_to_prompt at 0x7e4ce91c0160> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None base_url='https://53b5-34-125-224-48.ngrok-free.app' model='mistral' temperature=0.75 context_window=3900 request_timeout=300.0 prompt_key='prompt' additional_kwargs={}


Create a chat store to allow subsequent export operations on the chat content

In [ ]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()

memory = ChatMemoryBuffer.from_defaults(token_limit=30000,
                                        chat_store=chat_store,
                                        chat_store_key="user1",)

In [ ]:
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory
)

Let's start the interaction with Mistral

In [ ]:
input_text = """
Referring to Collect services from the context, please provide an Answer to the Question below.

Question: Within the scope of Collect services please find a read service for reading the content of a CSV file.
Answer:

"""

response = chat_engine.chat(input_text)

print(response)

 Based on the context information provided, you can find a read service for retrieving the content of a CSV file by making a GET request to the "/read-csv" endpoint in the Collect service for CSV data retrieval. The file path of the CSV file should be passed as a query parameter named "file\_path". Here is an example of how you can make this request using a web browser or a tool like Postman:

```
GET /read-csv?file_path=/content/data/your_csv_file.csv
```

This will return the content of the CSV file in the response, which is a binary string format (text/csv). If the file does not exist, you will receive a "404" error indicating that the CSV file was not found.


After chatting, persist the chat in a JSON file (if needed)

In [ ]:
chat_store_string = chat_store.persist(persist_path="./output_chat.json")

Use the following command to reset the chat

In [ ]:
chat_engine.reset()

**USE OF PromptTemplate CLASS**:
To build the prompt, resort to PromptTemplate class: prepare the template (using placeholders) and then, before interacting with the LLM, instantiate it

In [ ]:
from llama_index.core import PromptTemplate

prompt_template_qt1_string = """\
Referring to {atomic_service_type} from the provided context, please Answer the Question below.

Question: Within the scope of {atomic_service_type} please find a service for {task}
of {input_dataset}. \n
Answer:
"""

prompt_template_qt1 = PromptTemplate(prompt_template_qt1_string)

In [ ]:
inst_prompt_template_qt1 = prompt_template_qt1.format(
    atomic_service_type="COLLECT services",
    task="reading the content",
    input_dataset="a database table")

In [ ]:
print(inst_prompt_template_qt1)

Then, use this instantiated prompt to feed the LLM (as described before)